# Inception network

![](../img/inception_1.png)

Bức ảnh trên mô tả về cấu trúc của inception layer. Trong đó từ (28, 28, 192) input ta sẽ thực hiện đưa lần lượt input qua các conv và pool layer gồm:

- 1x1 conv same => output (28, 28, 64)
- 3x3 conv same => output (28, 28, 128)
- 5x5 conv same => ouput (28, 28, 32)
- max-pool (same) => output (28, 28 , 32)

Stack các kết quả lại với nhau được 1 outout có số chiều là (28, 28, 256)

## I. Vấn đề về computation cost

Xét việc áp 5x5 same conv lên input ta có:

![](../img/inception_2.png)

Ta cần sử dụng 32 bộ filter (5x5x192) để thực hiện convolution cho input (28x28x32)

=> cost = 28x28x32 * 5x5x192 = 120M phép nhân

Đây là 1 con số rất lớn.

Tuy nhiên ta có thể giảm thiếu cost trên bằng cách sử dụng thêm 1x1 same conv như sau:

![](../img/inception_3.png)

Bằng cách sử dụng 1x1 same conv (gọi là bottleneck layer) ta thấy rắng computation cost giảm xuống còn 12.4M

## II. Inception module

Giải quyết vấn đề về computation cost như trên cho nên ta sẽ không sử dụng convolution trực tiếp mà sẽ cho đi qua 1x1 same conv layer như sau:

![](../img/inception_4.png)

## III. GoogleNet

![](../img/inception_5.png)

Đây là cấu trúc của mạng GoogLeNet, ta thấy rằng mạng đang sử dụng nhiều Inception layer.
Bên cạnh đó ở một số hidden Inception layer còn có Fully connected layer và softmax => Để dựng đoán labels hay ta có thể nói răng việc sử dụng mạng này cho phép ta dự đoán labels ở bất kì layer nào. Việc sử dụng như vậy giúp ngăn việc overfitting




